In [1]:
import cv2
import os
import copy
import numpy as np
#from detector import Detectors
from skimage.filters import meijering
from skimage.morphology import watershed
from collections import defaultdict
import matplotlib.pyplot as plt
import pathlib
from scipy import ndimage as ndi
from skimage.feature import peak_local_max
from math import sqrt
import mahotas as mh
import pylab


In [2]:
def Sobel_gradient(blurred):
    gradX = cv2.Sobel(blurred, ddepth=cv2.CV_32F, dx=1, dy=0)
    gradY = cv2.Sobel(blurred, ddepth=cv2.CV_32F, dx=0, dy=1)
    gradient = cv2.subtract(gradX, gradY)
    gradient = cv2.convertScaleAbs(gradient)
    return gradX, gradY, gradient

def Thresh_and_blur(gradient):
    blurred = cv2.GaussianBlur(gradient, (9, 9), 0)
    (_, thresh) = cv2.threshold(blurred, 0, 255, cv2.THRESH_OTSU + cv2.THRESH_BINARY)
    return thresh

def image_morphology(thresh):
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (25, 25))
    closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    closed = cv2.erode(closed, None, iterations=4)
    closed = cv2.dilate(closed, None, iterations=4)
    return closed
def get_ID(contours):
    Id={}
    #count=0
    for c in contours:
        M = cv2.moments(c)
        print (M)
    return Id

In [3]:
# #method 1
# gamma = copy.deepcopy(gray)
# rows,cols = gamma.shape[0],gamma.shape[1]
# for i in range(rows):
#     for j in range(cols):
#         gamma[i][j]=10*pow(gamma[i][j],1.5)
# blurred = cv2.GaussianBlur(gamma, (9,9),0)
# ret, th = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
# holes = th.copy()
# _,_,gradient = Sobel_gradient(blurred)
# cv2.floodFill(holes,None, (0,0), 1) # 找到洞孔
# for i in range(th.shape[0]):
#     for j in range(th.shape[1]):
#         if holes[i,j]==0:
#             th[i,j]=255
# th = cv2.erode(th, None, iterations=4)
# img = th.copy()
# binary,contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL , cv2.CHAIN_APPROX_SIMPLE)
# cv2.drawContours(img,contours,-1,(0,0,255),6)

# for i in range(0,len(contours)):  
#     x, y, w, h = cv2.boundingRect(contours[i])   
#     cv2.rectangle(gamma, (x,y), (x+w,y+h), (153,153,0), 5) 
# plt.imshow(gamma,'gray')
# len(contours)

In [46]:
pre_center = {}
pre_area = {}
Id = {}
tracking = {}
#motion = {}
total_distance = {}
net_distance = {}
ratio = {}
mitosis = {}
count = {}
# path_seg = 'G:/20T2/COMP9517/Group Project/COMP9517 20T2 Group Project Image Sequences/Fluo-N2DL-HeLa/3_seg'
# path_box = 'G:/20T2/COMP9517/Group Project/COMP9517 20T2 Group Project Image Sequences/Fluo-N2DL-HeLa/3_box'
# path_track = 'G:/20T2/COMP9517/Group Project/COMP9517 20T2 Group Project Image Sequences/Fluo-N2DL-HeLa/3_track'
# path_motion = 'G:/20T2/COMP9517/Group Project/COMP9517 20T2 Group Project Image Sequences/Fluo-N2DL-HeLa/3_motion'
# path_count =  'G:/20T2/COMP9517/Group Project/COMP9517 20T2 Group Project Image Sequences/Fluo-N2DL-HeLa'

In [2]:
def  trajectory(tracking,try_img):
    for i in list(tracking.keys()):
        path = tracking[i]
        if len(path) == 1:
            continue
        else:
            for j in range(1,len(path)):
                cv2.line(try_img, path[j-1], path[j], (55,255,155),5)
    return try_img

In [3]:
def Euclidean_Distance(precenter,curcenter):
    dis = sqrt((precenter[0]-curcenter[0])**2+(precenter[1]-curcenter[1])**2)
    return dis
    

In [49]:
# method 2

def method2(gray,frame,pre_center,pre_area):
    cur_frame_total = {}
    cur_frame_net = {}
    cur_frame_ratio = {}
    motion = {}
    gamma = copy.deepcopy(gray)
    rows,cols = gamma.shape[0],gamma.shape[1]
    for i in range(rows):
        for j in range(cols):
            gamma[i][j]=10*pow(gamma[i][j],1.5)
    blurred = cv2.GaussianBlur(gamma, (9,9),0)
    #gray = cv2.cvtColor(try_img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (19,19),0)
    ret, th = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
#     pylab.imshow(blurred)
#     pylab.gray()
#     #pylab.jet()
#     pylab.show()
    holes = th.copy()
    #_,_,gradient = Sobel_gradient(blurred)
    cv2.floodFill(holes,None, (0,0), 1) # find hole
    for i in range(th.shape[0]):
        for j in range(th.shape[1]):
            if holes[i,j]==0:
                th[i,j]=255

    T = mh.thresholding.otsu(th)
    rmax = mh.regmax(blur)
    seeds,nr_nuclei = mh.label(rmax)
    #seeds = cv2.equalizeHist(seeds)

#     cv2.imwrite(f'{path_seg}/{frame}',
#                                 seeds.astype(np.uint8))
    dist = mh.distance(th>T)
    dist = dist.max() - dist
    dist -= dist.min()
    dist = dist/float(dist.ptp()) * 255
    dist = dist.astype(np.uint8)

    #print (nr_nuclei)
    #labeled,nr_objects = mh.label(seeds)
    #print (nr_objects)
    #nuclei = mh.cwatershed(dist,seeds)
    #ret, dist = cv2.threshold(dist, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    #dist = cv2.dilate(dist, None, iterations=2)
    #dist = cv2.equalizeHist(dist)


    seeds = seeds.astype(np.uint8)
    img = seeds.copy()
    binary,contours, hierarchy = cv2.findContours(img,  cv2.RETR_EXTERNAL , cv2.CHAIN_APPROX_SIMPLE)
    count[frame] = len(contours)
    #print(len(contours))
    cv2.drawContours(img,contours,-1,(0,0,255),6)
    index = 0

    center = {}
    area = {}
    #print (pre_center)
    #print (pre_area)
    if pre_center == {} and pre_area == {}:
        for i in range(0,len(contours)):  
            x, y, w, h = cv2.boundingRect(contours[i])
            cv2.rectangle(seeds, (x,y), (x+w,y+h), (125,125,0), 3)
            center[index] = (x+int(w/2),y+int(h/2))
            Id[index] = [frame]
            area[index] = w*h
            tracking[index] = [(x+int(w/2),y+int(h/2))]
            cv2.circle(seeds, (x+int(w/2), y+int(h/2)), 7, 128, -1)#绘制中心点
            #cv2.putText(seeds, f"{index}", (x+int(w/2) - 5, y+int(h/2) - 5),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            index += 1
    else:
        #print (1111111111111111)
        for i in range(0,len(contours)):
            x, y, w, h = cv2.boundingRect(contours[i])
            c = (x+int(w/2),y+int(h/2))
            a = w*h
            cv2.rectangle(seeds, (x,y), (x+w,y+h), (125,125,0), 3)
            cv2.circle(seeds, (x+int(w/2), y+int(h/2)), 7, 128, -1)#绘制中心点
            cost_min = 100000
            target = 0
            for j in list(pre_center.keys()):
                #pre_c = list(pre_center.values())
                distance = sqrt((c[0]-pre_center[j][0])**2+(c[1]-pre_center[j][1])**2)
                area_diff = abs(a-pre_area[j])
                #cost = 0.9*distance+0.1*area_diff
                cost = distance
                #print (cost)
                if cost < cost_min:
                    cost_min = cost
                    target = j
            #cv2.putText(gray, f"{target}", (pre_center[target][0] - 20,pre_center[target][1] - 20),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            if cost_min > 15:
                iden = list(Id.keys())[-1]+1
                Id[iden] = [frame]
                center[iden] = c
                area[iden] = a
                tracking[iden] = [c]
            else:
                if target not in list(center.keys()):
                    center[target] = c
                    area[target] = a
                    Id[target].append(frame)
                    #print (pre_center[target],c)
                    #cv2.line(seeds, pre_center[target], c, (125,125,0),10)
                    tracking[target].append(c)
                    if target in list(motion.keys()):
                        motion[target] = cost_min
                        pre_total = sum(total_distance[target])
                        total = pre_total+cost_min+1
                        total_distance[target].append(total)
                        cur_frame_total[target]= total
                        net = Euclidean_Distance(tracking[target][0],c)+1
                        net_distance[target].append(net)
                        cur_frame_net[target] = net
                        ratio[target].append(net/total)
                        cur_frame_ratio[target] = net/total
                    else:
                        motion[target] = cost_min
                        total_distance[target] = [cost_min]
                        cur_frame_total[target] = cost_min
                        net_distance[target] = [cost_min]
                        cur_frame_net[target] = cost_min
                        ratio[target] = [1]
                        cur_frame_ratio[target] = 1
                else:
                    iden = list(Id.keys())[-1]+1
                    Id[iden] = [frame]
                    center[iden] = c
                    area[iden] = a
                    tracking[iden] = [c]
                    if target not in list(mitosis.keys()):
                        mitosis[target] = frame
                        #print ("deplicate!!!!!!!!!s")
                    else:
                        if target == list(mitosis.keys())[-1]:
                            mitosis.pop(target)
#                             print ('wrong.....')
            
#     cv2.imwrite(f'{path_box}/{frame}',
#                                 seeds.astype(np.uint8))

    return center,area,cur_frame_total,cur_frame_net,cur_frame_ratio,motion

In [50]:
main_path = 'G:/20T2/COMP9517/Group Project/COMP9517 20T2 Group Project Image Sequences/'
dataset_1 = main_path + 'DIC-C2DH-HeLa'  
dataset_2 = main_path + 'Fluo-N2DL-HeLa'
dataset_3 = main_path + 'Phc-C2DL-PSC'

tiff_list = os.listdir('G:/20T2/COMP9517/Group Project/COMP9517 20T2 Group Project Image Sequences/Fluo-N2DL-HeLa/Sequence 3/')

tiff_list = tiff_list[1:]
#print (tiff_list)

In [51]:
main_path = 'G:/20T2/COMP9517/Group Project/COMP9517 20T2 Group Project Image Sequences/'
dataset_1 = main_path + 'DIC-C2DH-HeLa'  
dataset_2 = main_path + 'Fluo-N2DL-HeLa'
dataset_3 = main_path + 'Phc-C2DL-PSC'
#path_seg_dataset2 = 'G:/20T2/COMP9517/Group Project/COMP9517 20T2 Group Project Image Sequences/Fluo-N2DL-HeLa/Sequence_1_box/'
#try_list = ['t015.tif', 't016.tif']
for i in tiff_list:
    try_img = cv2.imread(f'G:/20T2/COMP9517/Group Project/COMP9517 20T2 Group Project Image Sequences/Fluo-N2DL-HeLa/Sequence 3/{i}',1)
    #try_img = cv2.imread('G:/20T2/COMP9517/Group Project/COMP9517 20T2 Group Project Image Sequences/PhC-C2DL-PSC/t000.tif',1)
    frame=i
    print(frame)
    gray = cv2.cvtColor(try_img, cv2.COLOR_BGR2GRAY)
    center,area,cur_frame_total,cur_frame_net,cur_frame_ratio,motion= method2(gray,frame,pre_center,pre_area)
    #print (motion)
#     file_total = open(f'{path_motion}/{frame}_total.txt', 'w') 
#     for k,v in cur_frame_total.items():
#         file_total.write(str(k)+' '+str(v)+'\n')
#     file_total.close()
#     file_net = open(f'{path_motion}/{frame}_net.txt', 'w') 
#     for k,v in cur_frame_net.items():
#         file_net.write(str(k)+' '+str(v)+'\n')
#     file_net.close()
#     file_ratio = open(f'{path_motion}/{frame}_ratio.txt', 'w') 
#     for k,v in cur_frame_ratio.items():
#         file_ratio.write(str(k)+' '+str(v)+'\n')
#     file_ratio.close()
#     file_motion = open(f'{path_motion}/{frame}_motion.txt', 'w') 
#     for k,v in motion.items():
#         file_motion.write(str(k)+' '+str(v)+'\n')
#     file_motion.close()
#     track = trajectory(tracking,try_img)
#     #track = cv2.cvtColor(track, cv2.COLOR_GRAY2BGR)
#     cv2.imwrite(f'{path_track}/{frame}',
#                             track)

    pre_center = center
    pre_area = area
# file_count = open(f'{path_count}/sequence2_count.txt', 'w') 
# for k,v in count.items():
#     file_count.write(str(k)+' '+str(v)+'\n')
# file_count.close()

t001.tif
t002.tif


H:\anacando\set\envs\py3.6\lib\site-packages\ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in true_divide


t003.tif
t004.tif
t005.tif
t006.tif
t007.tif
t008.tif
t009.tif
t010.tif
t011.tif
t012.tif
t013.tif
t014.tif
t015.tif
t016.tif
t017.tif
t018.tif
t019.tif
t020.tif
t021.tif
t022.tif
t023.tif
t024.tif
t025.tif
t026.tif
t027.tif
t028.tif
t029.tif
t030.tif
t031.tif
t032.tif
t033.tif
t034.tif
t035.tif
t036.tif
t037.tif
t038.tif
t039.tif
t040.tif
t041.tif
t042.tif
t043.tif
t044.tif
t045.tif
t046.tif
t047.tif
t048.tif
t049.tif
t050.tif
t051.tif
t052.tif
t053.tif
t054.tif
t055.tif
t056.tif
t057.tif
t058.tif
t059.tif
t060.tif
t061.tif
t062.tif
t063.tif
t064.tif
t065.tif
t066.tif
t067.tif
t068.tif
t069.tif
t070.tif
t071.tif
t072.tif
t073.tif
t074.tif
t075.tif
t076.tif
t077.tif
t078.tif
t079.tif
t080.tif
t081.tif
t082.tif
t083.tif
t084.tif
t085.tif
t086.tif
t087.tif
t088.tif
t089.tif
t090.tif
t091.tif


In [66]:
pre_center = {}
pre_area = {}
Id = {}
tracking = {}
# motion = {}
total_distance = {}
net_distance = {}
ratio = {}
mitosis = {}
count = {}
# path_seg = 'G:/20T2/COMP9517/Group Project/COMP9517 20T2 Group Project Image Sequences/Phc-C2DL-PSC/3_seg'
# path_box = 'G:/20T2/COMP9517/Group Project/COMP9517 20T2 Group Project Image Sequences/Phc-C2DL-PSC/3_box'
# path_track = 'G:/20T2/COMP9517/Group Project/COMP9517 20T2 Group Project Image Sequences/Phc-C2DL-PSC/3_track'
# path_motion = 'G:/20T2/COMP9517/Group Project/COMP9517 20T2 Group Project Image Sequences/Phc-C2DL-PSC/3_motion'
# path_count =  'G:/20T2/COMP9517/Group Project/COMP9517 20T2 Group Project Image Sequences/Phc-C2DL-PSC'

In [67]:
main_path = 'G:/20T2/COMP9517/Group Project/COMP9517 20T2 Group Project Image Sequences/'
dataset_1 = main_path + 'DIC-C2DH-HeLa'  
dataset_2 = main_path + 'Fluo-N2DL-HeLa'
dataset_3 = main_path + 'Phc-C2DL-PSC'

tiff_list = os.listdir('G:/20T2/COMP9517/Group Project/COMP9517 20T2 Group Project Image Sequences/Phc-C2DL-PSC/Sequence 3/')

tiff_list = tiff_list[1:]
#print(tiff_list)

In [68]:
# try_img2 = cv2.imread('G:/20T2/COMP9517/Group Project/COMP9517 20T2 Group Project Image Sequences/PhC-C2DL-PSC/t000.tif',1)
# gray2 = cv2.cvtColor(try_img2, cv2.COLOR_BGR2GRAY)
# plt.imshow(gray2,'gray')

In [69]:
# method 2
#try_img = cv2.imread('G:/20T2/COMP9517/Group Project/COMP9517 20T2 Group Project Image Sequences/Fluo-N2DL-HeLa/t081.tif',1)
def method2_2(gray2,frame,pre_center,pre_area):
    gamma2 = copy.deepcopy(gray2)
    cur_frame_total = {}
    cur_frame_net = {}
    cur_frame_ratio = {}
    #cur_count = {}
    motion = {}
    blur2 = cv2.GaussianBlur(gamma2, (9,9),0)
    ret, th2 = cv2.threshold(blur2, 170, 255, cv2.THRESH_BINARY)

    kernal = np.uint8(np.ones((3,3)))
    th2 = cv2.erode(th2, kernal, iterations=1)
    T2 = mh.thresholding.otsu(th2)
    th2 = th2>T2
    dist2 = mh.distance(th2>T2)
    dist2 = dist2.max() - dist2
    dist2 -= dist2.min()
    dist2 = dist2/float(dist2.ptp()) * 255
    dist2 = dist2.astype(np.uint8)
    rmax2 = mh.regmax(th2>T2)
    seeds2,nr_nuclei = mh.label(rmax2)
#     cv2.imwrite(f'{path_seg}/{frame}',
#                                 seeds2.astype(np.uint8))
    #print (nr_nuclei)
#     nuclei2 = mh.cwatershed(dist2,seeds2)
#     pylab.imshow(nuclei2)
#     pylab.gray()
#     pylab.jet()
#     pylab.show()
    seeds2 = seeds2.astype(np.uint8)
    #seeds2 = cv2.equalizeHist(seeds2)

    img2 = seeds2.copy()

    binary,contours, hierarchy = cv2.findContours(img2,  cv2.RETR_EXTERNAL , cv2.CHAIN_APPROX_SIMPLE)
    count[frame] = len(contours)
    cv2.drawContours(img2,contours,-1,(0,0,255),6)
    index = 0

    center = {}
    area = {}
    #print (pre_center)
    #print (pre_area)
    if pre_center == {} and pre_area == {}:
        for i in range(0,len(contours)):  
            x, y, w, h = cv2.boundingRect(contours[i])
            cv2.rectangle(seeds2, (x,y), (x+w,y+h), (125,125,0), 3)
            center[index] = (x+int(w/2),y+int(h/2))
            Id[index] = [frame]
            area[index] = w*h
            tracking[index] = [(x+int(w/2),y+int(h/2))]
            cv2.circle(seeds2, (x+int(w/2), y+int(h/2)), 7, 128, -1)#绘制中心点
            #cv2.putText(seeds, f"{index}", (x+int(w/2) - 5, y+int(h/2) - 5),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            index += 1
    else:
        #print (1111111111111111)
        for i in range(0,len(contours)):
            x, y, w, h = cv2.boundingRect(contours[i])
            c = (x+int(w/2),y+int(h/2))
            a = w*h
            cv2.rectangle(seeds2, (x,y), (x+w,y+h), (125,125,0), 3)
            cv2.circle(seeds2, (x+int(w/2), y+int(h/2)), 7, 128, -1)#绘制中心点
            cost_min = 100000
            target = 0
            for j in list(pre_center.keys()):
                #pre_c = list(pre_center.values())
                distance = sqrt((c[0]-pre_center[j][0])**2+(c[1]-pre_center[j][1])**2)
                area_diff = abs(a-pre_area[j])
                #cost = 0.9*distance+0.1*area_diff
                cost = distance
                #print (cost)
                if cost < cost_min:
                    cost_min = cost
                    target = j
            #cv2.putText(gray, f"{target}", (pre_center[target][0] - 20,pre_center[target][1] - 20),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
            if cost_min > 30:
                iden = list(Id.keys())[-1]+1
                Id[iden] = [frame]
                center[iden] = c
                area[iden] = a
                tracking[iden] = [c]
            else:
                if target not in list(center.keys()):
                    center[target] = c
                    area[target] = a
                    Id[target].append(frame)
                    #print (pre_center[target],c)
                    #cv2.line(seeds, pre_center[target], c, (125,125,0),10)
                    tracking[target].append(c)
                    if target in list(motion.keys()):
                        motion[target]=cost_min
                        pre_total = sum(total_distance[target])
                        total = pre_total+cost_min+1
                        total_distance[target].append(total)
                        cur_frame_total[target]= total
                        net = Euclidean_Distance(tracking[target][0],c)+1
                        net_distance[target].append(net)
                        cur_frame_net[target] = net
                        ratio[target].append(net/total)
                        cur_frame_ratio[target] = net/total
                    else:
                        motion[target] = cost_min
                        total_distance[target] = [cost_min]
                        cur_frame_total[target] = cost_min
                        net_distance[target] = [cost_min]
                        cur_frame_net[target] = cost_min
                        ratio[target] = [1]
                        cur_frame_ratio[target] = 1
                else:
                    iden = list(Id.keys())[-1]+1
                    Id[iden] = [frame]
                    center[iden] = c
                    area[iden] = a
                    tracking[iden] = [c]
                    if target not in list(mitosis.keys()):
                        mitosis[target] = frame
                        #print ("deplicate!!!!!!!!!s")
                    else:
                        if target == list(mitosis.keys())[-1]:
                            mitosis.pop(target)
                            #print ('wrong.....')

#     cv2.imwrite(f'{path_box}/{frame}',
#                                 seeds2.astype(np.uint8))
    return center,area,cur_frame_total,cur_frame_net,cur_frame_ratio,motion

In [70]:
for i in tiff_list:
    try_img = cv2.imread(f'G:/20T2/COMP9517/Group Project/COMP9517 20T2 Group Project Image Sequences/PhC-C2DL-PSC/Sequence 3/{i}',1)
    #try_img = cv2.imread('G:/20T2/COMP9517/Group Project/COMP9517 20T2 Group Project Image Sequences/PhC-C2DL-PSC/t000.tif',1)
    frame=i
    print(frame)
    gray = cv2.cvtColor(try_img, cv2.COLOR_BGR2GRAY)
    center,area,cur_frame_total,cur_frame_net,cur_frame_ratio,motion= method2_2(gray,frame,pre_center,pre_area)
#     file_total = open(f'{path_motion}/{frame}_total.txt', 'w') 
#     for k,v in cur_frame_total.items():
#         file_total.write(str(k)+' '+str(v)+'\n')
#     file_total.close()
#     file_net = open(f'{path_motion}/{frame}_net.txt', 'w') 
#     for k,v in cur_frame_net.items():
#         file_net.write(str(k)+' '+str(v)+'\n')
#     file_net.close()
#     file_ratio = open(f'{path_motion}/{frame}_ratio.txt', 'w') 
#     for k,v in cur_frame_ratio.items():
#         file_ratio.write(str(k)+' '+str(v)+'\n')
#     file_ratio.close()
#     file_motion = open(f'{path_motion}/{frame}_motion.txt', 'w') 
#     for k,v in motion.items():
#         file_motion.write(str(k)+' '+str(v)+'\n')
#     file_motion.close()
#     track = trajectory(tracking,try_img)
#     #track = cv2.cvtColor(track, cv2.COLOR_GRAY2BGR)
#     cv2.imwrite(f'{path_track}/{frame}',
#                             track)
    pre_center = center
    pre_area = area
# file_count = open(f'{path_count}/sequence3_count.txt', 'w') 
# for k,v in count.items():
#     file_count.write(str(k)+' '+str(v)+'\n')
# file_count.close()
  

t001.tif
t002.tif
t003.tif
t004.tif
t005.tif
t006.tif
t007.tif
t008.tif
t009.tif
t010.tif
t011.tif
t012.tif
t013.tif
t014.tif
t015.tif
t016.tif
t017.tif
t018.tif
t019.tif
t020.tif
t021.tif
t022.tif
t023.tif
t024.tif
t025.tif
t026.tif
t027.tif
t028.tif
t029.tif
t030.tif
t031.tif
t032.tif
t033.tif
t034.tif
t035.tif
t036.tif
t037.tif
t038.tif
t039.tif
t040.tif
t041.tif
t042.tif
t043.tif
t044.tif
t045.tif
t046.tif
t047.tif
t048.tif
t049.tif
t050.tif
t051.tif
t052.tif
t053.tif
t054.tif
t055.tif
t056.tif
t057.tif
t058.tif
t059.tif
t060.tif
t061.tif
t062.tif
t063.tif
t064.tif
t065.tif
t066.tif
t067.tif
t068.tif
t069.tif
t070.tif
t071.tif
t072.tif
t073.tif
t074.tif
t075.tif
t076.tif
t077.tif
t078.tif
t079.tif
t080.tif
t081.tif
t082.tif
t083.tif
t084.tif
t085.tif
t086.tif
t087.tif
t088.tif
t089.tif
t090.tif
t091.tif
t092.tif
t093.tif
t094.tif
t095.tif
t096.tif
t097.tif
t098.tif
t099.tif
t100.tif
t101.tif
t102.tif
t103.tif
t104.tif
t105.tif
t106.tif
t107.tif
t108.tif
t109.tif
t110.tif
t111.tif
t

In [ ]:
#method 1
# gamma2 = copy.deepcopy(gray2)
# # rows,cols = gamma.shape[0],gamma.shape[1]
# # for i in range(rows):
# #     for j in range(cols):
# #         gamma[i][j]=10*pow(gamma[i][j],1.5)
# blur2 = cv2.GaussianBlur(gamma2, (9,9),0)
# res,th2 = cv2.threshold(blur2, 170, 255, cv2.THRESH_BINARY)
# th2 = cv2.erode(th2, None, iterations=1)
# img2 = th2.copy()
# binary,contours, hierarchy = cv2.findContours(img2, cv2.RETR_EXTERNAL , cv2.CHAIN_APPROX_SIMPLE)
# print(len(contours))
# cv2.drawContours(img2,contours,-1,(0,0,255),6)
# #plt.imshow(img,'gray')
# for i in range(0,len(contours)):  
#     x, y, w, h = cv2.boundingRect(contours[i])   
#     cv2.rectangle(th2, (x,y), (x+w,y+h), (153,153,0), 5) 
# plt.imshow(th2,'gray')
# # len(contours)

In [20]:
pre_center = {}
pre_area = {}
Id = {}
tracking = {}
# motion = {}
total_distance = {}
net_distance = {}
ratio = {}
mitosis = {}
count = {}
#path_seg = 'G:/20T2/COMP9517/Group Project/COMP9517 20T2 Group Project Image Sequences/DIC-C2DH-HeLa/1_seg'
# path_box = 'G:/20T2/COMP9517/Group Project/COMP9517 20T2 Group Project Image Sequences/DIC-C2DH-HeLa/2_box'
# path_track = 'G:/20T2/COMP9517/Group Project/COMP9517 20T2 Group Project Image Sequences/DIC-C2DH-HeLa/2_track'
# path_motion = 'G:/20T2/COMP9517/Group Project/COMP9517 20T2 Group Project Image Sequences/DIC-C2DH-HeLa/2_motion'
# path_count =  'G:/20T2/COMP9517/Group Project/COMP9517 20T2 Group Project Image Sequences/DIC-C2DH-HeLa'

In [21]:
main_path = 'G:/20T2/COMP9517/Group Project/COMP9517 20T2 Group Project Image Sequences/'
dataset_1 = main_path + 'DIC-C2DH-HeLa'  
dataset_2 = main_path + 'Fluo-N2DL-HeLa'
dataset_3 = main_path + 'Phc-C2DL-PSC'

tiff_list = os.listdir('G:/20T2/COMP9517/Group Project/COMP9517 20T2 Group Project Image Sequences/DIC-C2DH-HeLa/1_seg/')

tiff_list = tiff_list[1:]
#print (tiff_list)

In [22]:
def method2_3(gray3,frame,pre_center,pre_area):
    gamma3 = copy.deepcopy(gray3)
    cur_frame_total = {}
    cur_frame_net = {}
    cur_frame_ratio = {}
    motion = {}
    blur3 = cv2.GaussianBlur(gamma3, (9,9),0)
    ret, th3 = cv2.threshold(blur3, 170, 255, cv2.THRESH_BINARY)
    binary,contours, hierarchy = cv2.findContours(th3,  cv2.RETR_EXTERNAL , cv2.CHAIN_APPROX_SIMPLE)
    count[frame] = len(contours)
    cv2.drawContours(th3,contours,-1,(0,0,255),6)
    index = 0

    center = {}
    area = {}
    #print (pre_center)
    #print (pre_area)
    if pre_center == {} and pre_area == {}:
        for i in range(0,len(contours)):  
            x, y, w, h = cv2.boundingRect(contours[i])
            cv2.rectangle(th3, (x,y), (x+w,y+h), (125,125,0), 3)
            center[index] = (x+int(w/2),y+int(h/2))
            Id[index] = [frame]
            area[index] = w*h
            tracking[index] = [(x+int(w/2),y+int(h/2))]
            cv2.circle(th3, (x+int(w/2), y+int(h/2)), 7, 128, -1)#绘制中心点
            #cv2.putText(th3, f"{index}", (x+int(w/2) - 5, y+int(h/2) - 5),cv2.FONT_HERSHEY_SIMPLEX, 3, (255, 255, 255), 3)
            index += 1
    else:
        #print (1111111111111111)
        for i in range(0,len(contours)):
            x, y, w, h = cv2.boundingRect(contours[i])
            c = (x+int(w/2),y+int(h/2))
            a = w*h
            
            cv2.rectangle(th3, (x,y), (x+w,y+h), (125,125,0), 3)
            cv2.circle(th3, (x+int(w/2), y+int(h/2)), 7, 128, -1)#绘制中心点
            cost_min = 100000
            target = 0
            for j in list(pre_center.keys()):
                #pre_c = list(pre_center.values())
                distance = sqrt((c[0]-pre_center[j][0])**2+(c[1]-pre_center[j][1])**2)
                area_diff = abs(a-pre_area[j])
                #cost = 0.6*distance+0.4*area_diff
                cost = 0.1*distance
                #print (area_diff)
                if cost < cost_min:
                    cost_min = cost
                    target = j
            #cv2.putText(th3, f"{target}", (pre_center[target][0],pre_center[target][1]),cv2.FONT_HERSHEY_SIMPLEX,3, (255, 255, 255), 3)
            #print (cost_min)
            if cost_min > 10:
                iden = list(Id.keys())[-1]+1
                Id[iden] = [frame]
                center[iden] = c
                area[iden] = a
                tracking[iden] = [c]
            else:
                if target not in list(center.keys()):
                    center[target] = c
                    area[target] = a
                    Id[target].append(frame)
                    #print (target,a)
                    #cv2.line(seeds, pre_center[target], c, (125,125,0),10)
                    tracking[target].append(c)
                    if target in list(motion.keys()):
                        motion[target]=cost_min
                        pre_total = sum(total_distance[target])
                        total = pre_total+cost_min+1
                        total_distance[target].append(total)
                        cur_frame_total[target]= total
                        net = Euclidean_Distance(tracking[target][0],c)+1
                        net_distance[target].append(net)
                        cur_frame_net[target] = net
                        ratio[target].append(net/total)
                        cur_frame_ratio[target] = net/total
                    else:
                        motion[target] = cost_min
                        total_distance[target] = [cost_min]
                        cur_frame_total[target] = cost_min
                        net_distance[target] = [cost_min]
                        cur_frame_net[target] = cost_min
                        ratio[target] = [1]
                        cur_frame_ratio[target] = 1
                else:
                    iden = list(Id.keys())[-1]+1
                    Id[iden] = [frame]
                    center[iden] = c
                    area[iden] = a
                    tracking[iden] = [c]
                    #print(iden,a)
                    #cv2.putText(th3, f"{iden}", (center[iden][0],center[iden][1]),cv2.FONT_HERSHEY_SIMPLEX,3, (255, 255, 255), 3)
                    if target not in list(mitosis.keys()):
                        if abs((area[target]+a) - pre_area[target]) <300 and abs(area[target]-a) <300:
                            mitosis[target] = frame
                            #print(area[target])
                            #print (pre_area[target],a)
                            #print ("deplicate!!!!!!!!!s")
                    else:
                        if target == list(mitosis.keys())[-1]:
                            mitosis.pop(target)
#     cv2.imwrite(f'{path_box}/{frame}',
#                                 th3.astype(np.uint8))
    return center,area,cur_frame_total,cur_frame_net,cur_frame_ratio

In [23]:
for i in tiff_list:
    try_img = cv2.imread(f'G:/20T2/COMP9517/Group Project/COMP9517 20T2 Group Project Image Sequences/DIC-C2DH-HeLa/1_seg/{i}',1)
    #try_img = cv2.imread('G:/20T2/COMP9517/Group Project/COMP9517 20T2 Group Project Image Sequences/PhC-C2DL-PSC/t000.tif',1)
    frame=i
    print(frame)
    gray = cv2.cvtColor(try_img, cv2.COLOR_BGR2GRAY)
    center,area,cur_frame_total,cur_frame_net,cur_frame_ratio= method2_3(gray,frame,pre_center,pre_area)
    print (mitosis)
#     file_total = open(f'{path_motion}/{frame}_total.txt', 'w') 
#     for k,v in cur_frame_total.items():
#         file_total.write(str(k)+' '+str(v)+'\n')
#     file_total.close()
#     file_net = open(f'{path_motion}/{frame}_net.txt', 'w') 
#     for k,v in cur_frame_net.items():
#         file_net.write(str(k)+' '+str(v)+'\n')
#     file_net.close()
#     file_ratio = open(f'{path_motion}/{frame}_ratio.txt', 'w') 
#     for k,v in cur_frame_ratio.items():
#         file_ratio.write(str(k)+' '+str(v)+'\n')
#     file_ratio.close()
#     file_motion = open(f'{path_motion}/{frame}_motion.txt', 'w') 
#     for k,v in motion.items():
#         file_motion.write(str(k)+' '+str(v)+'\n')
#     file_motion.close()
#     track = trajectory(tracking,try_img)
#     #track = cv2.cvtColor(track, cv2.COLOR_GRAY2BGR)
#     cv2.imwrite(f'{path_track}/{frame}',
#                             track)

    pre_center = center
    pre_area = area
# file_count = open(f'{path_count}/sequence2_count.txt', 'w') 
# for k,v in count.items():
#     file_count.write(str(k)+' '+str(v)+'\n')
# file_count.close()

t001.tif
{}
t002.tif
{}
t003.tif
{}
t004.tif
{}
t005.tif
{}
t006.tif
{}
t007.tif
{}
t008.tif
{}
t009.tif
{}
t010.tif
{}
t011.tif
{0: 't011.tif'}
t012.tif
{0: 't011.tif'}
t013.tif
{0: 't011.tif'}
t014.tif
{0: 't011.tif'}
t015.tif
{0: 't011.tif'}
t016.tif
{0: 't011.tif'}
t017.tif
{0: 't011.tif'}
t018.tif
{0: 't011.tif'}
t019.tif
{0: 't011.tif'}
t020.tif
{0: 't011.tif'}
t021.tif
{0: 't011.tif'}
t022.tif
{0: 't011.tif'}
t023.tif
{0: 't011.tif'}
t024.tif
{0: 't011.tif'}
t025.tif
{0: 't011.tif'}
t026.tif
{0: 't011.tif'}
t027.tif
{0: 't011.tif'}
t028.tif
{0: 't011.tif'}
t029.tif
{0: 't011.tif'}
t030.tif
{0: 't011.tif'}
t031.tif
{0: 't011.tif'}
t032.tif
{0: 't011.tif'}
t033.tif
{0: 't011.tif'}
t034.tif
{0: 't011.tif'}
t035.tif
{0: 't011.tif'}
t036.tif
{0: 't011.tif'}
t037.tif
{0: 't011.tif'}
t038.tif
{0: 't011.tif'}
t039.tif
{0: 't011.tif'}
t040.tif
{0: 't011.tif'}
t041.tif
{0: 't011.tif'}
t042.tif
{0: 't011.tif'}
t043.tif
{0: 't011.tif'}
t044.tif
{0: 't011.tif'}
t045.tif
{0: 't011.tif'}
t046.